In [ ]:
!pip uninstall -y cupy-cuda11x cupy-cuda12x cupy
!nvidia-smi
!pip install cupy-cuda11x
!apt-get install -y nvidia-cuda-toolkit
!pip install cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com

In [2]:
import dask.dataframe as dd
import dask_cudf
from dask.distributed import Client, LocalCluster
from sklearn.impute import KNNImputer
import cudf
import numpy as np
import pandas as pd
import sys

# Initialize a Dask LocalCluster to manage computations
cluster = LocalCluster(
    n_workers=2,  # Use two workers to maximize CPU and RAM usage
    threads_per_worker=2,  # 2 threads per worker
    memory_limit="6GB",  # Split memory between workers
    processes=True,  # Enable multiprocessing
)
client = Client(cluster)

# Example usage
filepath = '/content/drive/MyDrive/filtered_dbnsfp.csv'
output_filepath = 'dbnsfp_eda_results.txt'

# Missing Values list
na_list = [
    '#chr', 'pos(1-based)', 'ref', 'alt',
    'SIFT_converted_rankscore', 'SIFT4G_converted_rankscore', 'Polyphen2_HDIV_rankscore',
    '.', ''  # Shortened for clarity
]

dtypedict = {'#chr': 'object', 'pos(1-based)': 'object', 'ref': 'object', 'alt': 'object'}

# Load dbNSFP into Dask-cuDF (GPU-accelerated)
dbnsfp_data = dask_cudf.read_csv(filepath, na_values=na_list, dtype=dtypedict)

# Missing Value Analysis
rows, cols = dbnsfp_data.shape
with open(output_filepath, "w") as f:

    missing_values = dbnsfp_data.isnull().sum().compute()

    # Convert missing_values to Pandas Series to iterate over it
    missing_values = missing_values.to_pandas()

    # Now you can iterate over the missing_values
    with open(output_filepath, 'w') as f:
        f.write("Missing values per feature (sorted):\n")
        for col, count in missing_values.items():
            f.write(f"{col}: {count}\n")


    # Compute the threshold for dropping columns with more than 80% missing values
    threshold = int(0.8 * dbnsfp_data.shape[0].compute())  # Compute the number of rows

    # Drop columns with more than 80% missing values
    dbnsfp_data = dbnsfp_data.loc[:, dbnsfp_data.isnull().sum(axis=0) < threshold]

    # Now, dbnsfp_data will only include columns with fewer than 80% missing values


    # Imputation for numeric columns with <20% missing
    numeric_cols = dbnsfp_data.select_dtypes(include=['float64', 'int64']).columns
    imputer = KNNImputer(n_neighbors=5)
    dbnsfp_data[numeric_cols] = imputer.fit_transform(dbnsfp_data[numeric_cols].to_pandas())
    f.write(f"\nImputed columns: {numeric_cols}\n")

    # Save cleaned data
    dbnsfp_data.to_csv('/content/cleaned_dbnsfp.csv', single_file=True)

# GPU-specific operations
client.close()


/usr/local/lib/python3.10/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39693 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:33717
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:39693/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45857'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44769'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:44403', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:44403
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:51046
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:37685', name: 1, status: init, memory:

TypeError: DataFrame.dropna() got an unexpected keyword argument 'axis'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt

# File path
file_path = '/content/drive/MyDrive/filtered_dbnsfp.csv'
chunk_size = 100000  # Define the chunk size

#Missing Values list
na_list = [
    '#chr', 'pos(1-based)', 'ref', 'alt',
    'SIFT_converted_rankscore', 'SIFT4G_converted_rankscore', 'Polyphen2_HDIV_rankscore',
    'Polyphen2_HVAR_rankscore', 'LRT_converted_rankscore', 'MutationTaster_converted_rankscore',
    'MutationAssessor_rankscore', 'FATHMM_converted_rankscore', 'PROVEAN_converted_rankscore',
    'VEST4_rankscore', 'MetaSVM_rankscore', 'MetaLR_rankscore', 'MetaRNN_rankscore', 'M-CAP_rankscore',
    'REVEL_rankscore', 'MutPred_rankscore', 'MVP_rankscore', 'gMVP_rankscore', 'MPC_rankscore',
    'PrimateAI_rankscore', 'DEOGEN2_rankscore', 'BayesDel_addAF_rankscore', 'BayesDel_noAF_rankscore',
    'ClinPred_rankscore', 'LIST-S2_rankscore', 'VARITY_R_rankscore', 'VARITY_ER_rankscore',
    'VARITY_R_LOO_rankscore', 'VARITY_ER_LOO_rankscore', 'ESM1b_rankscore', 'EVE_rankscore',
    'AlphaMissense_rankscore', 'PHACTboost_rankscore', 'MutFormer_rankscore', 'MutScore_rankscore',
    'CADD_raw_rankscore', 'CADD_raw_rankscore_hg19', 'DANN_rankscore', 'fathmm-MKL_coding_rankscore',
    'fathmm-XF_coding_rankscore', 'Eigen-raw_coding_rankscore', 'Eigen-PC-raw_coding_rankscore',
    'GenoCanyon_rankscore', 'integrated_fitCons_rankscore', 'GM12878_fitCons_rankscore',
    'H1-hESC_fitCons_rankscore', 'HUVEC_fitCons_rankscore', 'LINSIGHT_rankscore', 'GERP++_RS_rankscore',
    'GERP_91_mammals_rankscore', 'phyloP100way_vertebrate_rankscore', 'phyloP470way_mammalian_rankscore',
    'phyloP17way_primate_rankscore', 'phastCons100way_vertebrate_rankscore', 'phastCons470way_mammalian_rankscore',
    'phastCons17way_primate_rankscore', 'SiPhy_29way_logOdds_rankscore', 'bStatistic_converted_rankscore', '.',''
]

# Initialize variables to aggregate results across chunks
missing_values_summary = None
data_cleaned_chunks = []
outlier_counts_summary = {}

# Define outlier detection function
def detect_outliers_iqr(df, feature):
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
    return outliers

# Process the data chunk by chunk
for chunk in pd.read_csv(file_path, na_values=na_list, chunksize=chunk_size):
    print(f"Processing chunk with shape: {chunk.shape}")

    # Set dtypes of index features to object
    index_features = ['#chr', 'pos(1-based)', 'ref', 'alt']
    for feature in index_features:
        if feature in chunk.columns:
            chunk[feature] = chunk[feature].astype('object')


    # Separate non-index features for EDA
    non_index_features = [col for col in chunk.columns if col not in index_features]
    chunk_non_index = chunk[non_index_features]

    # Count missing values per column in the chunk
    missing_values_chunk = chunk_non_index.isnull().sum()

    # Aggregate missing values across chunks
    if missing_values_summary is None:
        missing_values_summary = missing_values_chunk
    else:
        missing_values_summary += missing_values_chunk

    # Remove features with >= 80% missing values
    threshold = 0.8
    features_to_keep = missing_values_chunk[missing_values_chunk < threshold * chunk.shape[0]].index
    chunk_non_index = chunk_non_index[features_to_keep]

    # Impute missing values for features with <20% missing values
    imputer = KNNImputer(n_neighbors=5)
    chunk_imputed = pd.DataFrame(imputer.fit_transform(chunk_non_index), columns=chunk_non_index.columns)

    # Reattach index features
    chunk_imputed[index_features] = chunk[index_features].reset_index(drop=True)

    # Detect and remove outliers for numeric columns
    for feature in chunk_imputed.select_dtypes(include='number').columns:
        Q1 = chunk_imputed[feature].quantile(0.25)
        Q3 = chunk_imputed[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Count outliers for reporting
        outliers = chunk_imputed[(chunk_imputed[feature] < lower_bound) | (chunk_imputed[feature] > upper_bound)]
        outlier_counts_summary[feature] = outlier_counts_summary.get(feature, 0) + len(outliers)

        # Remove outliers
        chunk_imputed = chunk_imputed[(chunk_imputed[feature] >= lower_bound) & (chunk_imputed[feature] <= upper_bound)]

    # Collect the cleaned chunk
    data_cleaned_chunks.append(chunk_imputed)

# Combine all cleaned chunks
cleaned_data = pd.concat(data_cleaned_chunks, ignore_index=True)

# Report results
removed_features = missing_values_summary[missing_values_summary >= threshold * chunk_size].index
outlier_count_total = sum(outlier_counts_summary.values())

report = []
report.append(f"Initial missing values per feature:\n{missing_values_summary}")
report.append(f"Number of features removed due to missing values: {len(removed_features)}")
report.append(f"Outlier counts per feature:\n{outlier_counts_summary}")
report.append(f"Number of samples removed during outlier analysis: {outlier_count_total}")
report.append(f"Final size of the cleaned dataset: {cleaned_data.shape}")

# Write report to a text file
with open('dbnsfp_eda_results.txt', 'w') as report_file:
    report_file.write("\n\n".join(report))

print("Data cleaning report saved to 'dbnsfp_eda_results.txt'")

# Visualize box plots
features_with_outliers = [feature for feature, count in outlier_counts_summary.items() if count > 0][:2]
features_without_outliers = [feature for feature, count in outlier_counts_summary.items() if count == 0][:2]

# Plot box plots for features with outliers
for feature in features_with_outliers:
    sns.boxplot(x=cleaned_data[feature])
    plt.title(f"Box Plot for Feature with Outliers: {feature}")
    plt.show()

# Plot box plots for features without outliers
for feature in features_without_outliers:
    sns.boxplot(x=cleaned_data[feature])
    plt.title(f"Box Plot for Feature without Outliers: {feature}")
    plt.show()

# Save the cleaned data
cleaned_data.to_csv('cleaned_dbnsfp.csv', index=False)
print("Cleaned data saved to 'cleaned_dbnsfp.csv'")


Processing chunk with shape: (100000, 62)


KeyboardInterrupt: 